In [ ]:
# Eight point algorithm to compute the essential matrix
# Can also be used for homography and more

import numpy as np

# Outline of the 8-point algorithm
The fundamental matrix $F$ relates to camera views and is defined by the equation $\textbf{u'}^T F \textbf{u} = 0$, with $\textbf{u'}, \textbf{u}$ being point matchings between two images.

If we write $\textbf{u} = (u, v, 1)$ and $\textbf{u'} = (u', v', 1)$, then the equation can be flattened as

$$
\begin{equation}
uu' f_{11} + uv' f_{21} + u f_{31} + vu' f_{12} + vv' f_{22} + v f_{32} + u' f_{13} + v' f_{23} + f_{33} = 0
\end{equation}
$$

Therefore, a single point matching $\textbf{u'}, \textbf{u}$ yields one equation in the unknowns of $F$. We can represent this equation as the matrix $A$ that has line entries in the shape
$$
\begin{equation}
A_i = (uu', uv', u, vu', vv', v, u', v', 1)
\end{equation}

From 9 point matches, we obtain a set of linear equations of the form $Af = 0$.

## Constraints of $A$
Since $F$ is a projective transform, it is defined up to a scale. For this reason, and to avoid the trivial solution $f = 0$, we add the constraint $||f|| = 1$.

## Solving the system
Under these conditions, in the noise-free case 8 degrees of freedom remain, and thus 8 points are required to solve the system.
When noise is present, $f$ still has rank 9 and we seek a linear least squares solution.

Therefore we want to minimize $||Af||$, under the constraint $||f|| = f^Tf = 1$ that we defined above. The solution to this problem is the unit eigenvector of $A^TA$ corresponding to the smallest eigenvalue of $A$.

## Constraints of $F$
While $A$ has rank 9 (or 8), $F$ has rank 2. Furthermore, the left and right nullspaces of $F$ are generated by the epipoles in the two images.

Since the matrix obtained by solving the equations will not have rank 2, we need to enforce that constraint. To do so, we decompose $F$ as $F = U^TDU$ where $U$ is orthogonal and $D$ is $diag(r,s,t)$ with $r \ge s \ge t$, and then we project it into $F' = U^T diag(r, s, 0) U$, ie $F'$ is the closest matrix of rank 2 under the frobenius norm.

In [288]:
def normalize(points):
    """
    Normalizes / Pre-conditions the set of points for the DLT.
    The points are homogeneous.
    """
    T = np.eye(3)
    # Translation
    centroid = np.mean(points, 0)
    
    T[:, 2] -= centroid
    
    # Isotropic scaling
    mean_dist = np.linalg.norm(points[:,0:2] - centroid[0:2], axis=1).mean()
    factor = np.sqrt(2) / mean_dist
   
    T *= factor
    T[2,2] = 1

    return T

def test_normalize():
    points = np.random.random((10, 3))
    points[:, -1] = 1
    
    points += np.array([10,0,0])
    
    T = normalize(points)    
    
    points = (T @ points.T).T
    
    
    # Check scaling
    print(np.abs(np.linalg.norm(points[:,0:2],axis = 1).mean() - np.sqrt(2)) < 1e-6)
    # Check translation
    print(np.abs(np.mean(points[:, 0:2])) < 1e-6)
    
test_normalize()

True
True


In [289]:
def make_a_i(p1, p2):
    """
    Using one point correspondence, generates the corresponding
    two constraints in matrix form.
    Outputs a 1x9 matrix.
    """    
    return np.array(
        [p1[0] * p2[0], p1[0] * p2[1], p1[0], p1[1] * p2[0], p1[1] * p2[1], p1[1], p2[0], p2[1], 1]
    )

def eight_point(points_1, points_2):
    """

    """
    assert len(points_1) == len(points_2)
    
    T1 = normalize(points_1)
    T2 = normalize(points_2)
    
    points_1 = np.einsum("ij,kj->ki", T1, points_1)
    points_2 = np.einsum("ij,kj->ki", T2, points_2)
    
           
    A = np.zeros((len(points_1), 9))
    for i, correspondance in enumerate(zip(points_1, points_2)):
        # 1. Compute matrix A by stacking the A_i
        A[i] = make_a_i(*correspondance)
        
    # 2. Compute SVD of A
    u, s, v = np.linalg.svd(A)

    print(s)
    E = v[-1].reshape((3, 3)).T
    E = (T2).T @ E @ T1
    
    return E 

In [290]:
# create image points
#points = np.random.random((9, 3))
#points[:, -1] = 1

# transform them
from scipy.spatial.transform import Rotation as R
r = R.from_euler('zyx', [90, 45, 30], degrees=True)
rot = r.as_matrix()

points2 = (rot @ points.T).T
points2 /= points2[:, None, -1]

E = eight_point(points, points2)

[6.72147636e+00 6.06475878e+00 5.00981325e+00 3.41926307e+00
 1.80141202e+00 1.43677175e+00 1.13633509e-15 3.32924103e-16
 1.80899758e-16]


In [291]:
for i in range(points.shape[0]):
    print(points2[i].T @ E @ points[i] )

3.3306690738754696e-16
4.440892098500626e-16
-1.1102230246251565e-16
2.220446049250313e-16
4.440892098500626e-16
0.0
-4.440892098500626e-16
0.0
1.1102230246251565e-16


In [292]:
import cv2 as cv
Ep, _ = cv.findEssentialMat(points[:,0:2], points2[:,0:2], np.eye(3))
for i in range(points.shape[0]):
    print(points2[i].T @ Ep @ points[i] )

0.0019700010578235227
-0.031590621454848256
-0.011823496556750968
-5.551115123125783e-17
-5.551115123125783e-17
8.326672684688674e-17
-1.1102230246251565e-16
0.009472295528601793
-8.326672684688674e-17
